In [2]:
import config
import pandas as pd

In [3]:
data = pd.read_csv(config.CRAWLED_DATA)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160667 entries, 0 to 1160666
Data columns (total 32 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1160667 non-null  int64  
 1   ID                          1160667 non-null  int64  
 2   Case Number                 1160667 non-null  object 
 3   Date                        1160667 non-null  object 
 4   Block                       1160667 non-null  object 
 5   IUCR                        1160667 non-null  object 
 6   Primary Type                1160667 non-null  object 
 7   Description                 1160667 non-null  object 
 8   Location Description        1154139 non-null  object 
 9   Arrest                      1160667 non-null  bool   
 10  Domestic                    1160667 non-null  bool   
 11  Beat                        1160667 non-null  int64  
 12  District                    1160667 non-null  float64
 1

In [4]:
# --- FEATURE SELECTION ---
necessary_columns = [
    'ID', 'Case Number', 'Date', 'Primary Type', 'Description',
    'Location Description', 'Arrest', 'Domestic', 'District',
    'Ward', 'Community Area', 'Year', 'Month', 'Latitude', 'Longitude'
]
cleaned_data = data[necessary_columns]

In [5]:
# --- DROP UNECCESSARY VALUES IN PRIMARY TYPE ---

typeToDrop = {
    'NON-CRIMINAL',
    'OTHER OFFENSE',
    'OTHER NARCOTIC VIOLATION',
    'OBSCENITY',
    'PUBLIC INDECENCY',
    'CONCEALED CARRY LICENSE VIOLATION',
    'LIQUOR LAW VIOLATION',
    'GAMBLING',
    'RITUALISM',
}
cleaned_data = cleaned_data[~cleaned_data['Primary Type'].isin(typeToDrop)]
cleaned_data.shape

(1160667, 15)

In [6]:
# --- ADD THE COUNT OF PRIMARY TYPE INTO THE DATAFRAME ---
cleaned_data["primary_type_count"] = cleaned_data.groupby("Primary Type")["Primary Type"].transform("count")
cleaned_data["primary_type_count"]

0            9676
1           31555
2          141930
3          268613
4            6241
            ...  
1160662    110690
1160663     90187
1160664    110690
1160665     90187
1160666    141930
Name: primary_type_count, Length: 1160667, dtype: int64

In [7]:
# --- ADD THE TOTAL CRIMES PER YEAR ---

cleaned_data["total_crimes_per_yr"] = cleaned_data.groupby("Year")["Year"].transform("count")
cleaned_data["total_crimes_per_yr"]

0          224691
1          246748
2          246748
3          246748
4          195093
            ...  
1160662     54103
1160663     54103
1160664     54103
1160665     54103
1160666     54103
Name: total_crimes_per_yr, Length: 1160667, dtype: int64

In [14]:
# --- ADDING MONTHLY TOTAL ---

monthly_counts = cleaned_data.groupby(["Year", "Month"]).size().reset_index(name="monthly_crime_count")
cleaned_data = cleaned_data.merge(monthly_counts, on=["Year", "Month"], how="left")
cleaned_data["monthly_crime_count"] = cleaned_data["monthly_crime_count_y"]
cleaned_data.drop(columns=["monthly_crime_count_x", "monthly_crime_count_y"], inplace=True)
cleaned_data

,ID,Case Number,Date,Primary Type,Description,Location Description,Arrest,Domestic,District,Ward,Community Area,Year,Month,Latitude,Longitude,primary_type_count,total_crimes_per_yr,monthly_crime_count
0,13311263,JG503434,2022-07-29 03:39:00,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,RESIDENCE,True,False,10.0,25.0,30.0,2022,7,NaN,NaN,9676,224691,20979
1,13053066,JG103252,2023-01-03 16:44:00,NARCOTICS,MANUFACTURE / DELIVER - CRACK,SIDEWALK,True,False,11.0,28.0,26.0,2023,1,NaN,NaN,31555,246748,19935
2,13203321,JG415333,2023-09-06 17:00:00,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,1.0,42.0,32.0,2023,9,41.886018,-87.633938,141930,246748,21305
3,13204489,JG416325,2023-09-06 11:00:00,THEFT,OVER $500,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,1.0,4.0,32.0,2023,9,41.871835,-87.626151,268613,246748,21305
4,12419690,JE295655,2021-07-07 10:30:00,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,RESIDENCE,False,False,5.0,10.0,54.0,2021,7,41.655116,-87.594883,6241,195093,18040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160662,13802159,JJ212459,2025-04-09 22:30:00,ASSAULT,SIMPLE,CHA PARKING LOT / GROUNDS,False,False,2.0,4.0,38.0,2025,4,41.805127,-87.608894,110690,54103,4949
1160663,13801723,JJ213223,2025-04-09 16:45:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,18.0,42.0,8.0,2025,4,41.893923,-87.639581,90187,54103,4949
1160664,13801213,JJ212469,2025-04-09 23:00:00,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,RESIDENCE,False,True,15.0,37.0,25.0,2025,4,41.891120,-87.762457,110690,54103,4949
1160665,13806474,JJ218891,2025-03-31 04:00:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,SIDEWALK,False,False,18.0,42.0,8.0,2025,3,41.893180,-87.634686,90187,54103,17655


In [15]:
# Function to generalize all the specific location description
def generalize_location(loc_desc):
  # Handle none values
  if pd.isna(loc_desc):
    return "Other"
  loc_desc = str(loc_desc).upper()
  # Residential
  if any(keyword in loc_desc for keyword in ["RESIDENCE", "APARTMENT", "HOUSE", "CHA APARTMENT", "RESIDENTIAL", "PORCH", "YARD", "GARAGE", "BASEMENT", "VESTIBULE", "HALLWAY"]):
      return "Residential"
  # Public Transportation
  elif any(keyword in loc_desc for keyword in ["CTA", "TRAIN", "BUS", "SUBWAY", "PLATFORM", "STATION", "RAILROAD"]):
      return "Public Transportation"
  # Commercial
  elif any(keyword in loc_desc for keyword in ["STORE", "RESTAURANT", "BAR ", "TAVERN", "HOTEL", "MOTEL", "BANK", "CURRENCY EXCHANGE", "GAS STATION", "OFFICE", "WAREHOUSE", "FACTORY", "LIQUOR", "BARBER", "BEAUTY SALON", "PAWN SHOP", "ATM", "CASINO"]):
      return "Commercial"
  # Street/Outdoor
  elif any(keyword in loc_desc for keyword in ["STREET", "SIDEWALK", "ALLEY", "PARKING LOT", "VACANT LOT", "PARK PROPERTY", "LAKEFRONT", "RIVERBANK", "FOREST PRESERVE", "BRIDGE", "HIGHWAY", "GANGWAY", "DRIVEWAY"]):
      return "Street/Outdoor"
  # Institutional
  elif any(keyword in loc_desc for keyword in ["SCHOOL", "HOSPITAL", "CHURCH", "SYNAGOGUE", "GOVERNMENT", "LIBRARY", "POLICE", "FIRE STATION", "NURSING HOME", "DAY CARE", "COLLEGE", "UNIVERSITY"]):
      return "Institutional"
  # Airport
  elif any(keyword in loc_desc for keyword in ["AIRPORT", "AIRCRAFT"]):
      return "Airport"
  # Vehicle
  elif any(keyword in loc_desc for keyword in ["VEHICLE", "AUTO", "TAXICAB", "UBER", "LYFT", "TRUCK", "BOAT"]):
      return "Vehicle"
  # Other
  else:
      return "Other"


cleaned_data["generalized_loc"] = cleaned_data["Location Description"].apply(generalize_location)
cleaned_data

,ID,Case Number,Date,Primary Type,Description,Location Description,Arrest,Domestic,District,Ward,Community Area,Year,Month,Latitude,Longitude,primary_type_count,total_crimes_per_yr,monthly_crime_count,generalized_loc
0,13311263,JG503434,2022-07-29 03:39:00,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,RESIDENCE,True,False,10.0,25.0,30.0,2022,7,NaN,NaN,9676,224691,20979,Residential
1,13053066,JG103252,2023-01-03 16:44:00,NARCOTICS,MANUFACTURE / DELIVER - CRACK,SIDEWALK,True,False,11.0,28.0,26.0,2023,1,NaN,NaN,31555,246748,19935,Street/Outdoor
2,13203321,JG415333,2023-09-06 17:00:00,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,1.0,42.0,32.0,2023,9,41.886018,-87.633938,141930,246748,21305,Residential
3,13204489,JG416325,2023-09-06 11:00:00,THEFT,OVER $500,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,1.0,4.0,32.0,2023,9,41.871835,-87.626151,268613,246748,21305,Residential
4,12419690,JE295655,2021-07-07 10:30:00,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,RESIDENCE,False,False,5.0,10.0,54.0,2021,7,41.655116,-87.594883,6241,195093,18040,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160662,13802159,JJ212459,2025-04-09 22:30:00,ASSAULT,SIMPLE,CHA PARKING LOT / GROUNDS,False,False,2.0,4.0,38.0,2025,4,41.805127,-87.608894,110690,54103,4949,Street/Outdoor
1160663,13801723,JJ213223,2025-04-09 16:45:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,18.0,42.0,8.0,2025,4,41.893923,-87.639581,90187,54103,4949,Residential
1160664,13801213,JJ212469,2025-04-09 23:00:00,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,RESIDENCE,False,True,15.0,37.0,25.0,2025,4,41.891120,-87.762457,110690,54103,4949,Residential
1160665,13806474,JJ218891,2025-03-31 04:00:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,SIDEWALK,False,False,18.0,42.0,8.0,2025,3,41.893180,-87.634686,90187,54103,17655,Street/Outdoor


In [16]:
cleaned_data.drop(columns=["Location Description"], inplace=True)
cleaned_data

,ID,Case Number,Date,Primary Type,Description,Arrest,Domestic,District,Ward,Community Area,Year,Month,Latitude,Longitude,primary_type_count,total_crimes_per_yr,monthly_crime_count,generalized_loc
0,13311263,JG503434,2022-07-29 03:39:00,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,True,False,10.0,25.0,30.0,2022,7,NaN,NaN,9676,224691,20979,Residential
1,13053066,JG103252,2023-01-03 16:44:00,NARCOTICS,MANUFACTURE / DELIVER - CRACK,True,False,11.0,28.0,26.0,2023,1,NaN,NaN,31555,246748,19935,Street/Outdoor
2,13203321,JG415333,2023-09-06 17:00:00,CRIMINAL DAMAGE,TO VEHICLE,False,False,1.0,42.0,32.0,2023,9,41.886018,-87.633938,141930,246748,21305,Residential
3,13204489,JG416325,2023-09-06 11:00:00,THEFT,OVER $500,False,False,1.0,4.0,32.0,2023,9,41.871835,-87.626151,268613,246748,21305,Residential
4,12419690,JE295655,2021-07-07 10:30:00,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,False,False,5.0,10.0,54.0,2021,7,41.655116,-87.594883,6241,195093,18040,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160662,13802159,JJ212459,2025-04-09 22:30:00,ASSAULT,SIMPLE,False,False,2.0,4.0,38.0,2025,4,41.805127,-87.608894,110690,54103,4949,Street/Outdoor
1160663,13801723,JJ213223,2025-04-09 16:45:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,18.0,42.0,8.0,2025,4,41.893923,-87.639581,90187,54103,4949,Residential
1160664,13801213,JJ212469,2025-04-09 23:00:00,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,False,True,15.0,37.0,25.0,2025,4,41.891120,-87.762457,110690,54103,4949,Residential
1160665,13806474,JJ218891,2025-03-31 04:00:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,18.0,42.0,8.0,2025,3,41.893180,-87.634686,90187,54103,17655,Street/Outdoor


In [17]:
# --- TOTAL COUNT BY LONGITUDE AND LATTITUDE ---

cleaned_data["Lat_round"] = cleaned_data["Latitude"].round(3)
cleaned_data["Lon_round"] = cleaned_data["Longitude"].round(3)
location_counts = cleaned_data.groupby(["Lat_round", "Lon_round"]).size().reset_index(name="location_crime_count")
cleaned_data = cleaned_data.merge(location_counts, on=["Lat_round", "Lon_round"], how="left")
cleaned_data

,ID,Case Number,Date,Primary Type,Description,Arrest,Domestic,District,Ward,Community Area,...,Month,Latitude,Longitude,primary_type_count,total_crimes_per_yr,monthly_crime_count,generalized_loc,Lat_round,Lon_round,location_crime_count
0,13311263,JG503434,2022-07-29 03:39:00,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,True,False,10.0,25.0,30.0,...,7,NaN,NaN,9676,224691,20979,Residential,NaN,NaN,NaN
1,13053066,JG103252,2023-01-03 16:44:00,NARCOTICS,MANUFACTURE / DELIVER - CRACK,True,False,11.0,28.0,26.0,...,1,NaN,NaN,31555,246748,19935,Street/Outdoor,NaN,NaN,NaN
2,13203321,JG415333,2023-09-06 17:00:00,CRIMINAL DAMAGE,TO VEHICLE,False,False,1.0,42.0,32.0,...,9,41.886018,-87.633938,141930,246748,21305,Residential,41.886,-87.634,278.0
3,13204489,JG416325,2023-09-06 11:00:00,THEFT,OVER $500,False,False,1.0,4.0,32.0,...,9,41.871835,-87.626151,268613,246748,21305,Residential,41.872,-87.626,141.0
4,12419690,JE295655,2021-07-07 10:30:00,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,False,False,5.0,10.0,54.0,...,7,41.655116,-87.594883,6241,195093,18040,Residential,41.655,-87.595,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160662,13802159,JJ212459,2025-04-09 22:30:00,ASSAULT,SIMPLE,False,False,2.0,4.0,38.0,...,4,41.805127,-87.608894,110690,54103,4949,Street/Outdoor,41.805,-87.609,79.0
1160663,13801723,JJ213223,2025-04-09 16:45:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,18.0,42.0,8.0,...,4,41.893923,-87.639581,90187,54103,4949,Residential,41.894,-87.640,25.0
1160664,13801213,JJ212469,2025-04-09 23:00:00,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,False,True,15.0,37.0,25.0,...,4,41.891120,-87.762457,110690,54103,4949,Residential,41.891,-87.762,13.0
1160665,13806474,JJ218891,2025-03-31 04:00:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,18.0,42.0,8.0,...,3,41.893180,-87.634686,90187,54103,17655,Street/Outdoor,41.893,-87.635,234.0


In [18]:
# cleaned_data.drop(columns=["Latitude", "Longitude"], inplace=True)
cleaned_data.to_csv("datasets/cleaned_crime_data.csv", index=False)
# cleaned_data